# SDM

In [68]:
using SpeciesDistributionToolkit
using CairoMakie
using Statistics
CairoMakie.activate!(px_per_unit = 6.0)

In [ ]:
place = SpeciesDistributionToolkit.gadm("PRY")
extent = SpeciesDistributionToolkit.boundingbox(place)

In [ ]:
provider = RasterData(CHELSA2, BioClim)

In [ ]:
L = SDMLayer{Float16}[SDMLayer(provider; layer=l, extent...) for l in layers(provider)]

In [ ]:
mask!(L, place)

In [ ]:
tx = taxon("Akodon montensis")
presences = occurrences(tx, first(L), "occurrenceStatus" => "PRESENT", "country" => "PY", "limit" => 300, "hasCoordinate" => true)
occurrences!(presences)

In [ ]:
presencelayer = mask(first(L), presences)
background = pseudoabsencemask(DistanceToEvent, presencelayer)
bgpoints = backgroundpoints(nodata(background, d -> d < 5), 8sum(presencelayer))

In [ ]:
heatmap(L[1], colormap=:Blues)
lines!(place[1].geometry, color=:black)
scatter!(presencelayer, color=:white, strokecolor=:black, strokewidth=2, markersize=10, label="Virtual presences")
scatter!(bgpoints, color=:white, strokecolor=:grey, strokewidth=1, markersize=7, label="Pseudo-absences")
current_figure()

In [ ]:
sdm = SDM(PCATransform, DecisionTree, L, presencelayer, bgpoints)

In [ ]:
folds = kfold(sdm);
cv = crossvalidate(sdm, folds; threshold = true);
mean(mcc.(cv.validation))

In [ ]:
forwardselection!(sdm, folds)
cv2 = crossvalidate(sdm, folds; threshold = true);
mean(mcc.(cv2.validation))

In [ ]:
variables(sdm)

In [ ]:
train!(sdm)

In [ ]:
heatmap(predict(sdm, L; threshold=false), colormap=:Greens)
lines!(place[1].geometry, color=:black)
scatter!(presencelayer, color=:white, strokecolor=:black, strokewidth=2, markersize=10, label="Virtual presences")
scatter!(bgpoints, color=:white, strokecolor=:grey, strokewidth=1, markersize=7, label="Pseudo-absences")
current_figure()

In [ ]:
ensemble = Bagging(sdm, 25)
bagfeatures!(ensemble)

In [ ]:
train!(ensemble)

In [ ]:
heatmap(predict(ensemble, L; threshold=false), colormap=:Greens)
lines!(place[1].geometry, color=:black)
scatter!(presencelayer, color=:white, strokecolor=:black, strokewidth=2, markersize=10, label="Virtual presences")
scatter!(bgpoints, color=:white, strokecolor=:grey, strokewidth=1, markersize=7, label="Pseudo-absences")
current_figure()

In [ ]:
#| label: fig-sdm-output
#| fig-cap: "also yeah"
heatmap(predict(ensemble, L; threshold=true), colormap=:Greens)
lines!(place[1].geometry, color=:black)
scatter!(presencelayer, color=:white, strokecolor=:black, strokewidth=2, markersize=10, label="Virtual presences")
scatter!(bgpoints, color=:white, strokecolor=:grey, strokewidth=1, markersize=7, label="Pseudo-absences")
current_figure()